In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
#Load original taxonomy
vse = pd.read_csv('VSE Research Portfolio.csv')

In [3]:
#Split Category by certian characters then expand dataframe using explode
new = vse['Category'].str.split("; and")
vse['Category'] = new
vse = vse.explode('Category')
new = vse['Category'].str.split(";")
vse['Category'] = new
vse = vse.explode('Category')
new = vse['Category'].str.split("/")
vse['Category'] = new
vse = vse.explode('Category')
vse['Category'] = vse['Category'].str.strip()
vse

,Topic,Area,Category
0,Digital Systems and Data,Artificial Intelligence,Cognitive Systems
0,Digital Systems and Data,Artificial Intelligence,Intelligent Agents
1,Digital Systems and Data,Artificial Intelligence,Knowledge Representation and Reasoning
2,Digital Systems and Data,Artificial Intelligence,Machine Learning
3,Digital Systems and Data,Artificial Intelligence,Natural Language Processing
...,...,...,...
130,Specific Business Sectors,Transportation,Safety and Security
130,Specific Business Sectors,Transportation,Emergency Response
131,Specific Business Sectors,Transportation,Sustainability and Resilience
132,Specific Business Sectors,Transportation,System Management


In [4]:
#Split Area by certian characters then expand dataframe using explode
new = vse['Area'].str.split("; and")
vse['Area'] = new
vse = vse.explode('Area')
new = vse['Area'].str.split(";")
vse['Area'] = new
vse = vse.explode('Area')
vse['Area'] = vse['Area'].str.strip()
vse

,Topic,Area,Category
0,Digital Systems and Data,Artificial Intelligence,Cognitive Systems
0,Digital Systems and Data,Artificial Intelligence,Intelligent Agents
1,Digital Systems and Data,Artificial Intelligence,Knowledge Representation and Reasoning
2,Digital Systems and Data,Artificial Intelligence,Machine Learning
3,Digital Systems and Data,Artificial Intelligence,Natural Language Processing
...,...,...,...
130,Specific Business Sectors,Transportation,Safety and Security
130,Specific Business Sectors,Transportation,Emergency Response
131,Specific Business Sectors,Transportation,Sustainability and Resilience
132,Specific Business Sectors,Transportation,System Management


In [5]:
#Set unique ID values for each Topic, Area, and Category
from collections import defaultdict

temp = defaultdict(lambda: len(temp))
Topic_ID = [temp[ele] for ele in vse['Topic']]
Topic_ID = [str(x) for x in Topic_ID]
Topic_ID = ["t" + s for s in Topic_ID]
temp = defaultdict(lambda: len(temp))
Area_ID = [temp[ele] for ele in vse['Area']]
Area_ID = [str(x) for x in Area_ID]
Area_ID = ["a" + s for s in Area_ID]
temp = defaultdict(lambda: len(temp))
Category_ID = [temp[ele] for ele in vse['Category']]
Category_ID = [str(x) for x in Category_ID]
Category_ID = ["c" + s for s in Category_ID]

vse['Topic_ID'] = Topic_ID
vse['Area_ID'] = Area_ID
vse['Category_ID'] = Category_ID
#Adjust Capitalization
vse['Topic'] = vse['Topic'].str.lower()
vse['Area'] = vse['Area'].str.lower()
vse['Category'] = vse['Category'].str.lower()
vse

,Topic,Area,Category,Topic_ID,Area_ID,Category_ID
0,digital systems and data,artificial intelligence,cognitive systems,t0,a0,c0
0,digital systems and data,artificial intelligence,intelligent agents,t0,a0,c1
1,digital systems and data,artificial intelligence,knowledge representation and reasoning,t0,a0,c2
2,digital systems and data,artificial intelligence,machine learning,t0,a0,c3
3,digital systems and data,artificial intelligence,natural language processing,t0,a0,c4
...,...,...,...,...,...,...
130,specific business sectors,transportation,safety and security,t5,a43,c158
130,specific business sectors,transportation,emergency response,t5,a43,c159
131,specific business sectors,transportation,sustainability and resilience,t5,a43,c160
132,specific business sectors,transportation,system management,t5,a43,c161


In [6]:
#Save expanded dictionary
vse.to_csv('VSE Research Portfolio Explode.csv')


In [4]:
#Load other files to obtain other stated research interests
CEC = pd.read_csv('loadable_profiles_final_all.csv')
GS = pd.read_csv('Scholar_GMU_Names.csv')
GS_pubs = pd.read_csv('GoogleScholar Final Dataset.csv')

In [13]:
#Adjust names and take neccessary columns
CEC['names'] = CEC['Name']
CEC_temp = CEC[['ID', 'names', 'Research Interest']]

In [6]:
#Search for names in CEC names that aren't in the Google Scholar names
unique_names = GS_pubs['authors'].unique()
GS = GS[['names', 'topics']]
GS
common_names = GS.loc[GS['names'].isin(unique_names)]
df = pd.DataFrame(unique_names, columns = ['names'])
missing_names = df.loc[~df['names'].isin(common_names['names'])]
missing_names

,names
0,abolfazl safikhani
1,aditya johri
2,ahmed bashatah
3,ahmed bin zaman
4,alexander brodsky
...,...
153,yotam gingold
154,zhisheng yan
155,ziwei zhu
156,ziyu yao


In [7]:
#Create Dataframe with research interests from Google Scholar topics and CEC stated research interests
common = pd.merge(common_names, CEC_temp, on = 'names')
missing = pd.merge(missing_names, CEC_temp, on = 'names')
common_and_missing = common.append(missing, ignore_index = True)
common_and_missing = common_and_missing.sort_values(by=['ID'])
common_and_missing = common_and_missing.rename(columns={'topics': 'Google Scholar topics'})
common_and_missing = common_and_missing.reset_index()
common_and_missing = common_and_missing.drop(columns=['index'])
common_and_missing

C:\Users\Simon\AppData\Local\Temp\ipykernel_10468\3365348463.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  common_and_missing = common.append(missing, ignore_index = True)


,Google Scholar topics,ID,names,Research Interest
0,NaN,0,ariela sofer,"optimization,"
1,NaN,3,sanjeev setia,NaN
2,NaN,5,abolfazl safikhani,NaN
3,NaN,8,david kepplinger,"robust statistics for high-dimensional data, r..."
4,NaN,13,pramita bagchi,NaN
...,...,...,...,...
125,NaN,271,linghan zhang,NaN
126,NaN,272,md tanvir arafin,NaN
127,NaN,273,mingkui wei,NaN
128,NaN,276,moinul hossain,NaN


In [14]:
#Combine CEC research interests and google Scholar research interests, adjust capitalization and remove nan, , 
CEC_GS = pd.merge(CEC, common_and_missing, on = ['ID','names','Research Interest'])
CEC_GS = CEC_GS.drop(columns=['names'])
CEC_GS['All_Research_Interest'] = CEC_GS['Research Interest'].astype(str) + ',' + CEC_GS['Google Scholar topics']
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].replace('nan,','', regex=True)
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].replace(',,',',', regex=True)
#CEC_GS = CEC_GS.replace({np.nan: 'None'})
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].astype(str).str.lower()
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].str.split(",")
#CEC_GS = CEC_GS.replace({np.nan: 'N/A'})
CEC_GS

,Unnamed: 0,ID,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link,Google Scholar topics,All_Research_Interest
0,0,0,ariela sofer,"associate dean, administration and faculty aff...",703-993-1692,ariela sofer is interested in nonlinear progr...,"optimization,","phd, operations research, george washington u...",https://cec.gmu.edu/profiles/asofer,NaN,http://mason.gmu.edu/~asofer/,NaN,[nan]
1,3,3,sanjeev setia,professor\ncomputer science\nschool of computi...,703-993-4098,sanjeev setia is a professor of computer scien...,NaN,NaN,https://cec.gmu.edu/profiles/setia,setia@gmu.edu,http://cs.gmu.edu/~setia/\n,NaN,[nan]
2,5,5,abolfazl safikhani,assistant professor,NaN,abolfazl safikhani is currently an assistant ...,NaN,"phd, statistics, michigan state university\n",https://cec.gmu.edu/profiles/asafikha,asafikha@gmu.edu,https://sites.google.com/site/abolfazlsafikhan...,NaN,[nan]
3,8,8,david kepplinger,assistant professor,NaN,david is an assistant professor of statistics...,"robust statistics for high-dimensional data, r...","phd, statistics, university of british columbi...",https://statistics.gmu.edu/node/301,dkepplin@gmu.edu,https://www.dkepplinger.org\n,NaN,[nan]
4,13,13,pramita bagchi,assistant professor,703-993-1674,pramita bagchi’s research focuses on modellin...,NaN,NaN,https://cec.gmu.edu/profiles/pbagchi,pbagchi@gmu.edu,https://pramitabagchi.squarespace.com/\n,NaN,[nan]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,271,271,linghan zhang,assistant professor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan]
126,272,272,md tanvir arafin,"assistant professor, cyber security engineering",NaN,tanvir arafin's work focuses on hardware secu...,NaN,NaN,https://cec.gmu.edu/profiles/marafin,marafin@gmu.edu,NaN,NaN,[nan]
127,273,273,mingkui wei,"assistant professor, cyber security engineering",NaN,mingkui wei is an assistant professor in the ...,NaN,"phd, computing engineering, north carolina sta...",https://cec.gmu.edu/profiles/mwei2,mwei2@gmu.edu,https://mason.gmu.edu/~mwei2/\n,NaN,[nan]
128,276,276,moinul hossain,"assistant professor, cyber security engineering",NaN,dr. hossain is an assistant professor in the ...,NaN,"phd, electrical engineering, university of nor...",https://cec.gmu.edu/profiles/mhossa5,mhossa5@gmu.edu,mhossa5@gmu.edu\nhttps://moinulhossain.com\nht...,NaN,[nan]


In [15]:
#Remove reparted values in All_Research_Interests
test = CEC_GS['All_Research_Interest']
test
new =[]
for i in test:
    if isinstance(i, list):
        res = [*set(i)]
        new.append(res)
    else:
        new.append(np.nan)


#Expand All research interests so each interests have its own row
CEC_GS['All_Research_Interest'] = new
#CEC_GS = CEC_GS.replace({np.nan: 'N/A'})
#temp = CEC_GS['All_Research_Interest'].str.strip()
CEC_GS = CEC_GS.explode('All_Research_Interest')
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].str.strip()
CEC_GS = CEC_GS.replace({np.nan: 'N/A'})
CEC_GS
#temp

,Unnamed: 0,ID,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link,Google Scholar topics,All_Research_Interest
0,0,0,ariela sofer,"associate dean, administration and faculty aff...",703-993-1692,ariela sofer is interested in nonlinear progr...,"optimization,","phd, operations research, george washington u...",https://cec.gmu.edu/profiles/asofer,N/A,http://mason.gmu.edu/~asofer/,N/A,nan
1,3,3,sanjeev setia,professor\ncomputer science\nschool of computi...,703-993-4098,sanjeev setia is a professor of computer scien...,N/A,N/A,https://cec.gmu.edu/profiles/setia,setia@gmu.edu,http://cs.gmu.edu/~setia/\n,N/A,nan
2,5,5,abolfazl safikhani,assistant professor,N/A,abolfazl safikhani is currently an assistant ...,N/A,"phd, statistics, michigan state university\n",https://cec.gmu.edu/profiles/asafikha,asafikha@gmu.edu,https://sites.google.com/site/abolfazlsafikhan...,N/A,nan
3,8,8,david kepplinger,assistant professor,N/A,david is an assistant professor of statistics...,"robust statistics for high-dimensional data, r...","phd, statistics, university of british columbi...",https://statistics.gmu.edu/node/301,dkepplin@gmu.edu,https://www.dkepplinger.org\n,N/A,nan
4,13,13,pramita bagchi,assistant professor,703-993-1674,pramita bagchi’s research focuses on modellin...,N/A,N/A,https://cec.gmu.edu/profiles/pbagchi,pbagchi@gmu.edu,https://pramitabagchi.squarespace.com/\n,N/A,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,271,271,linghan zhang,assistant professor,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,nan
126,272,272,md tanvir arafin,"assistant professor, cyber security engineering",N/A,tanvir arafin's work focuses on hardware secu...,N/A,N/A,https://cec.gmu.edu/profiles/marafin,marafin@gmu.edu,N/A,N/A,nan
127,273,273,mingkui wei,"assistant professor, cyber security engineering",N/A,mingkui wei is an assistant professor in the ...,N/A,"phd, computing engineering, north carolina sta...",https://cec.gmu.edu/profiles/mwei2,mwei2@gmu.edu,https://mason.gmu.edu/~mwei2/\n,N/A,nan
128,276,276,moinul hossain,"assistant professor, cyber security engineering",N/A,dr. hossain is an assistant professor in the ...,N/A,"phd, electrical engineering, university of nor...",https://cec.gmu.edu/profiles/mhossa5,mhossa5@gmu.edu,mhossa5@gmu.edu\nhttps://moinulhossain.com\nht...,N/A,nan


In [16]:
#Remove spaces from column names and replace with _
CEC_GS = CEC_GS.rename(columns={'Phone Number': 'Phone_Number', 'Research Interest': 'Research_Interest', 
                               'Profile Link': 'Profile_Link', 'Personal Link': 'Personal_Link',
                               'Google Scholar topics': 'Google_Scholar_topics'})

In [17]:
#Savle files
common.to_csv('common_names.csv')
missing.to_csv('missing_names.csv')
common_and_missing.to_csv('common_and_missing.csv')
CEC_GS.to_csv('CEC_GS.csv')

In [237]:
#Take all areas and categories in current dictionary
vse['Topic_l'] = vse['Topic'].str.lower()
vse['Area_l'] = vse['Area'].str.lower()
vse['Category_l'] = vse['Category'].str.lower()
Area_unique = vse['Area_l'].unique()
Cat_unique = vse['Category_l'].unique()


In [238]:
#Create dataframe that holds all All_Reseach_Interests that exist within  areas of dictionary; Save to file
in_area = CEC_GS[CEC_GS['All_Research_Interest'].isin(Area_unique)].copy()
print(len(in_area))
area_ids = []
for i in in_area['All_Research_Interest']:
    
    area_ids.append(vse.loc[vse['Area'] == i, 'Area_ID'].unique()[0])

area_ids
#in_area['Area_ID'] = ""
in_area['Area_ID'] = list(area_ids)
in_area
in_area.to_csv('in_area.csv')

41


In [239]:
#Create dataframe that holds all All_Reseach_Interests that exist within  categoires of dictionary; Save to file
in_cat = CEC_GS[CEC_GS['All_Research_Interest'].isin(Cat_unique)].copy()
print(len(in_cat))
cat_ids = []
for i in in_cat['All_Research_Interest']:
    
    cat_ids.append(vse.loc[vse['Category'] == i, 'Category_ID'].unique()[0])


#in_area['Area_ID'] = ""
in_cat['Category_ID'] = list(cat_ids)
in_cat
in_cat.to_csv('in_cat.csv')

54


In [241]:
#Create dataframe that holds all All_Reseach_Interests that dont exist within  categoires or areas of dictionary
#Save to file
not_in_cat = CEC_GS[~CEC_GS['All_Research_Interest'].isin(Cat_unique)].copy()
not_in_area = CEC_GS[~CEC_GS['All_Research_Interest'].isin(Area_unique)].copy()
not_in_area
s1 = pd.merge(not_in_cat, not_in_area, how='inner', on=list(not_in_area.columns.values))
s1.to_csv('not_in_cat_or_area.csv')